<a href="https://colab.research.google.com/github/facebookresearch/esm/blob/main/examples/inverse_folding/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inverse folding with ESM-IF1

The ESM-IF1 inverse folding model is built for predicting protein sequences from their backbone atom coordinates. We provide examples here 1) to sample sequence designs for a given structure and 2) to score sequences for a given structure.

Trained with 12M protein structures predicted by AlphaFold2, the ESM-IF1 model consists of invariant geometric input processing layers followed by a sequence-to-sequence transformer, and achieves 51% native sequence recovery on structurally held-out backbones. The model is also trained with span masking to tolerate missing backbone coordinates and therefore can predict sequences for partially masked structures.

See [GitHub README](https://github.com/facebookresearch/esm/tree/main/examples/inverse_folding) for the complete user guide, and see our [bioRxiv pre-print](https://doi.org/10.1101/2022.04.10.487779) for more details.

## Environment setup (colab)
This step might take up to 10 minutes the first time. 

If using a local jupyter environment, instead of the following, we recommend configuring a conda environment upon first use in command line:
```
conda create -n inverse python=3.9
conda activate inverse
conda install pytorch cudatoolkit=11.3 -c pytorch
conda install pyg -c pyg -c conda-forge
conda install pip
pip install biotite
pip install git+https://github.com/facebookresearch/esm.git
```

Afterwards, `conda activate inverse` to activate this environment before starting `jupyter notebook`.

Below is the setup for colab notebooks:

We recommend using GPU runtimes on colab (Menu bar -> Runtime -> Change runtime type -> Hardware accelerator -> GPU)

In [1]:
# Colab environment setup

# Install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

# Install esm
!pip install -q git+https://github.com/facebookresearch/esm.git

# Install biotite
!pip install -q biotite

     |████████████████████████████████| 7.9 MB 16.8 MB/s 
     |████████████████████████████████| 3.5 MB 27.7 MB/s 
     |████████████████████████████████| 2.5 MB 35.7 MB/s 
     |████████████████████████████████| 750 kB 25.8 MB/s 
     |████████████████████████████████| 407 kB 30.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 31.8 MB 1.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


### Verify that pytorch-geometric is correctly installed

If the notebook crashes at the import, there is likely an issue with the version of torch_geometric and torch_sparse being incompatible with the torch version.

In [2]:
## Verify that pytorch-geometric is correctly installed
import torch_geometric
import torch_sparse
from torch_geometric.nn import MessagePassing

## Load model
This steps takes a few minutes for the model to download.

**UPDATE**: It is important to set the model in eval mode through `model = model.eval()` to disable random dropout for optimal performance.

In [3]:
import esm
model, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
model = model.eval()

/home/chloehsu/inverse_opensource/esm_public_fork/esm/esm/pretrained.py:172: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


## Load structure from PDB or CIF files

As an example, let's look at Golgi Casein Kinase, the [PDB Molecule of the Month from January 2022](https://pdb101.rcsb.org/motm/265).

Milk is a complex mixture of proteins, fats, and nutrients that provides everything that a growing infant needs. Most of the protein in cow’s milk is casein, whereas human milk has lesser amounts of casein. 

The Golgi casein kinase (PDB entry 5YH2) adds phosphates to casein and also to many other types of secreted proteins. It is most active as a complex of two similar types of proteins. Fam20C is the catalytic subunit. It binds to casein and transfers a phosphate from ATP to the protein.

In this example, let's focus on chain C (the catalytic subunit Fam20C).

You may also upload your own CIF or PDB file and specify the chain id.

In [4]:
!wget https://files.rcsb.org/download/5YH2.cif -P data/    # save this to the data folder in colab

--2022-05-27 12:53:21--  https://files.rcsb.org/download/5YH2.cif
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.222
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘data/5YH2.cif.1’

5YH2.cif.1              [ <=>                ]   1.39M  --.-KB/s    in 0.09s   

2022-05-27 12:53:21 (15.5 MB/s) - ‘data/5YH2.cif.1’ saved [1456779]



Load chain C from this CIF file:

In [5]:
fpath = 'data/5YH2.cif' # .pdb format is also acceptable
chain_id = 'C'
structure = esm.inverse_folding.util.load_structure(fpath, chain_id)
coords, native_seq = esm.inverse_folding.util.extract_coords_from_structure(structure)
print('Native sequence:')
print(native_seq)

Found 6 chains: ['A' 'C' 'B' 'D' 'A' 'B'] 

Loaded chain C

Native sequence:
SVLQSLFEHPLYRTVLPDLTEEDTLFNLNAEIRLYPKAASESYPNWLRFHIGINRYELYSRHNPVIAALLRDLLSQKISSVGMKSGGTQLKLIMSFQNYGQALFKPMKQTREQETPPDFFYFSDFERHNAEIAAFHLDRILDFRRVPPVAGRLVNMTREIRDVTRDKKLWRTFFVSPANNICFYGECSYYCSTEHALCGKPDQIEGSLAAFLPDLALAKRKTWRNPWRRSYHKRKKAEWEVDPDYCDEVKQTPPYDRGTRLLDIMDMTIFDFLMGNMDRHHYETFEKFGNDTFIIHLDNGRGFGKHSHDEMSILVPLTQCCRVKRSTYLRLQLLAKEEYKLSSLMEESLLQDRLVPVLIKPHLEALDRRLRLVLKVLSDCVEKDGFSAVVENDLD


Visualize chain C in this CIF file:

In [6]:
!pip install -q py3Dmol

In [7]:
try:
    import py3Dmol

    def view_pdb(fpath, chain_id):
        with open(fpath) as ifile:
            system = "".join([x for x in ifile])

        view = py3Dmol.view(width=600, height=400)
        view.addModelsAsFrames(system)
        view.setStyle({'model': -1, 'chain': chain_id}, {"cartoon": {'color': 'spectrum'}})
        view.zoomTo()
        view.show()

except ImportError:
    def view_pdb(fpath, chain_id):
        print("Install py3Dmol to visualize, or use pymol")

In [8]:
view_pdb(fpath, chain_id)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Sample sequence and calculate sequence recovery:

In [9]:
import numpy as np

sampled_seq = model.sample(coords, temperature=1)
print('Sampled sequence:', sampled_seq)

recovery = np.mean([(a==b) for a, b in zip(native_seq, sampled_seq)])
print('Sequence recovery:', recovery)

Sampled sequence: STLKQWFHSPLTLNDDPYSSAEDVLFSPTRVLSLLPALQGRDLPDFILFWTGIDKFHMYKIDSPVISSLRKRLRTERVREVQQLMGGSKLVLKFTFEDLGAAMFKPVAASTSEETPPHWEFWEVKKTSRAVVASWHLDYLFQLKSTAPSAGRVLNLVRDIRDVTSDSQLQSTFVQTPSRQLCFYGSNTFRSNLRDAICGNRDEIWGSVIAELPHANVALRDVYTSPWKRSKTKFKISLWRTKPDYAKYIRLTPLFKEGDNFLELQKAFIFDYLQGNDDRAKWVAFQKFGVDQELLICDNGAGFGRFDHLDMEILSPLQQGAYFSQNLYDHVIALQSSDYSMSRLMKSVLKADERYPILSEPYLLQLDTRLETVRKILDDCVREIGSNNSLIVEKL
Sequence recovery: 0.3468354430379747


In [10]:
# Lower sampling temperature typically results in higher sequence recovery but less diversity

sampled_seq = model.sample(coords, temperature=1e-6)
print('Sampled sequence:', sampled_seq)

recovery = np.mean([(a==b) for a, b in zip(native_seq, sampled_seq)])
print('Sequence recovery:', recovery)

Sampled sequence: ERLRAWWASPLTQLPDPGLSEEDLLFDPEELLALLPEEEEEELPAWLRFWTGIRRRRLYERESPDVEELLRRLRTARVRRVGQKSGGRSLVLRFEFEDLGSAGFKPRVAELDEETPPEWGFWEVLQRARAVVAAYRLDRLLDLRQVPPAAGRRLDLVTELRDVTDDEELRSTFFVTPEEELCFYGRCEFRCDREHALCGRPDVVEGALVAELPDERIAPRGVYLNPWAHARERDVEALWEVDPDYCEYVRRLPPFREGRLLLELANAYVFDFLMGNADRHTFSTFERFGLDTFLLLLDNGFGFGRADYLDERILRPLEQCCLLSERLYRRLLALSEEEFSLEELMEEELGRDELWPVLARPFLRQLDRRLRRVLEVLEECEEECGREEVLVREEG
Sequence recovery: 0.4582278481012658


## Conditional sequence log-likelihoods for given backbone coordinates

The log-likelihood scores could be used to predict mutational effects. See also our [script](https://github.com/facebookresearch/esm/tree/main/examples/inverse_folding#scoring-sequences) for batch scoring,

In [11]:
ll_fullseq, ll_withcoord = esm.inverse_folding.util.score_sequence(model, alphabet, coords, native_seq)

print(f'average log-likelihood on entire sequence: {ll_fullseq:.2f} (perplexity {np.exp(-ll_fullseq):.2f})')
print(f'average log-likelihood excluding missing coordinates: {ll_withcoord:.2f} (perplexity {np.exp(-ll_withcoord):.2f})')

average log-likelihood on entire sequence: -1.61 (perplexity 5.01)
average log-likelihood excluding missing coordinates: -1.61 (perplexity 5.01)


## Masking part of the backbone coordinates
To partially mask backbone coordinates, simply set the masked coordinates to np.inf. 

Typically, the sequence perplexity will be higher on masked regions than on unmasked regions.

In [12]:
from copy import deepcopy
masked_coords = deepcopy(coords)
masked_coords[:15] = np.inf # mask the first 10 residues
ll_fullseq, ll_withcoord = esm.inverse_folding.util.score_sequence(model, alphabet, masked_coords, native_seq)

print(f'average log-likelihood on entire sequence: {ll_fullseq:.2f} (perplexity {np.exp(-ll_fullseq):.2f})')
print(f'average log-likelihood excluding missing coordinates: {ll_withcoord:.2f} (perplexity {np.exp(-ll_withcoord):.2f})')

average log-likelihood on entire sequence: -1.64 (perplexity 5.16)
average log-likelihood excluding missing coordinates: -1.59 (perplexity 4.92)


## Extract encoder output as structure representation
The encoder output may also be used as a representation for the structure.

For a set of input coordinates with L amino acids, the encoder output will have shape L x 512.

In [13]:
rep = esm.inverse_folding.util.get_encoder_output(model, alphabet, coords)
len(coords), rep.shape

(395, torch.Size([395, 512]))